<strong>Date :</strong> Créé le 03 Avril 2021| Mis à jour le 11 Avril 2021 </strong>

<strong>Compétition Kaggle - Team Théo
    
@auteur : </strong>Théo SACCAREAU & Théo VEDIS

<strong>(2-2)_features_text_2
      
Description :</strong> A travers ce Notebook, nous poursuivrons l'analyse du contenu et l'extraction de features textuelles en réalisant une analyse topicale.


Temps d'exécution du Notebook : environ  1h30.

<strong> (!) L'exécution de ce Notebook est FACULTATIVE.(!) </strong>Aucune feature n'y sera générée, vous pouvez directement passer au Notebook (2-3)_features_network_1. 

# Installer / Télécharger / Importer des librairies

In [ ]:
# Librairies usuelles
import pandas as pd
import numpy as np
from tqdm import tqdm 

# Librairie pour le texte
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import gensim
from gensim import corpora
from gensim.models import LdaModel, CoherenceModel

# Chemin 

In [3]:
# Chemin relatif vers le dossier "data" (inutile de le changer).
pathFile = "../data/" 

# Chargement des données d'entrée

In [5]:
# Chargement du fichier .csv contenant les contenus textuels nettoyés.
# Temps exécution : environ 40s
data_LDA = pd.read_csv(pathFile + "data_LDA.csv")

In [6]:
# On remplace les valeurs nulles par des chaînes vides. 
data_LDA = data_LDA['bodyNLP2'].fillna('')

# (3) Analyse topicale
L'objectif ici est de réaliser une fouille thématique non surpervisée, c'est-à-dire trouver les sujets / les thèmes évoqués dans une collection de texte. On va donc réaliser du clustering en regroupant dans des groupes des documents (ici les commentaires/posts) similaires. 

Plusieurs modèles sont possibles (kmeans, LSA, PLSA, NMF, etc), nous avons fait le choix d'utiliser le modèle LDA : Allocation Dirichlet Latente. C'est le modèle qui offre les meilleurs résultats et le plus utilisé. 

Sans revenir sur tout ce qui a été vu en cours, ce modèle essaie de modéliser comment une personne écrit un document : elle choisit un topic puis elle sélectionne des mots à l'intérieur de ce topic. La notion de distribution est donc au centre de ce modèle : la distribution de mots dans chaque thématique, dans chaque document, etc. Dernier point, en entré ce modèle prend uniquement une matrice de fréquence (pas de TF-IDF). 

In [7]:
# On récupère les informations scrapées sur les topics.
# Temps exécution : 40s
df2 = pd.read_json(pathFile + "data_post.json").T 
topics = df2['title'] 

On applique au contenu des posts le même prétraitement que celui appliqué aux contenus des commentaires (tokenisation, suppression des stopwords et lemmatisation). 

In [8]:
# (1) Tokenisation 
# On ne prend en compte que les caractères alpha-numérique, la ponctuation
# est donc exclue.
tokenizer = nltk.RegexpTokenizer(r'\w+')

# (2) Suppression des mots vides
stop_words = set(tuple(nltk.corpus.stopwords.words('english')))
topics = topics.apply(lambda x : " ".join([i for i in tokenizer.tokenize(x.lower()) if not i in stop_words and len(i) >= 4]))

# (3) Lemmatisation 
wnl = WordNetLemmatizer()
topics = [" ".join([wnl.lemmatize(words) for words in tokenizer.tokenize(x)]) for x in tqdm(topics)]

100%|██████████| 148848/148848 [00:04<00:00, 31907.35it/s]


In [9]:
# On concatène la liste des contenus des commentaires à la liste des 
# contenus des posts.  
texts = [tokenizer.tokenize(comment) for comment in tqdm(data_LDA)]
texts.extend( [tokenizer.tokenize(topic) for topic in tqdm(topics)])

100%|██████████| 148848/148848 [00:00<00:00, 413728.18it/s]


On prépare les données que nous donnerons au modèle LDA.

In [10]:
# Créer le dictionnaire qui associe un identifiant à chaque mot 
# Temps exécution : environ 2 min
id2word  = corpora.Dictionary(texts)

# Créer le corpus
corpus = [id2word.doc2bow(text) for text in tqdm(texts)]

100%|██████████| 4383818/4383818 [00:54<00:00, 81110.11it/s] 


On met en place le modèle LDA. 

In [11]:
# Construire le modèle LDA

# Choix du nombre de topic totalement arbitraire (cf ci-dessous)
num_topic = 20 

# Temps exécution : 50 min. 
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, 
                                            id2word=id2word, 
                                            num_topics=num_topic, 
                                            chunksize=len(corpus), 
                                            update_every = 1, 
                                            random_state=1,
                                            alpha='auto')

Observons le résultat.

In [12]:
# Affichage des mots les plus probables pour chaque topic
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.018*"people" + 0.011*"think" + 0.007*"time" + 0.007*"like" + 0.007*"know"')
(1, '0.013*"people" + 0.009*"time" + 0.009*"would" + 0.009*"know" + 0.008*"much"')
(2, '0.015*"like" + 0.010*"really" + 0.008*"good" + 0.006*"never" + 0.006*"people"')
(3, '0.013*"time" + 0.013*"would" + 0.012*"like" + 0.010*"year" + 0.007*"people"')
(4, '0.017*"know" + 0.012*"people" + 0.007*"make" + 0.007*"like" + 0.007*"back"')
(5, '0.018*"like" + 0.010*"people" + 0.009*"thing" + 0.008*"would" + 0.008*"make"')
(6, '0.015*"like" + 0.012*"people" + 0.009*"year" + 0.006*"feel" + 0.006*"thing"')
(7, '0.022*"like" + 0.009*"would" + 0.007*"please" + 0.007*"time" + 0.007*"thing"')
(8, '0.009*"people" + 0.009*"would" + 0.008*"like" + 0.007*"make" + 0.005*"shit"')
(9, '0.012*"like" + 0.010*"time" + 0.009*"would" + 0.008*"want" + 0.007*"think"')
(10, '0.016*"like" + 0.013*"people" + 0.012*"would" + 0.009*"time" + 0.005*"even"')
(11, '0.010*"like" + 0.008*"think" + 0.007*"thing" + 0.006*"shit" + 0.006*"could"')


A la vue des résultats, le modèle de LDA donne clairement des résultats pas satisfaisants. Plusieurs éléments peuvent expliquer cela. 

Tout d'abord le choix du nombre de topic, nous avons pris 20 purement arbitrairement. Mais avec plus de 150 000 links/posts et 4 millions de commentaires, on aurait dû sûrement augmenter cette valeur. Le problème c'est qu'au delà (50 par exemple), la RAM ne suportait pas. Normalement, pour choisir le nombre de cluster optimal, nous devrions effectuer plusieurs fois le LDA pour des valeurs de k différentes et comparer les résultats des métriques (perplexity ou coherence). Cependant, avec plus de 50 min pour entrainer un seul modèle, nous sommes limités par le temps. 

Ensuite, les paramètres du modèle peuvent avoir jouer un rôle. Par exemple, nous avons laissé la valeur par défaut 1 pour le paramètre `passes` (nombre de passages dans le corpus pendant la formation), or beaucoup de sources indiquent que ce paramètre doit être mis à 10 voire 20. Toujours limité par les temps de calcul, nous ne pouvions mettre une telle valeur. C'est ce qui peut expliquer les mauvais résultats. 

Enfin, dernière explication, peut-être que notre traitement de texte préalable n'était pas suffisant.

Il ne semble clairement pas utile d'en déduire des features. Initialement, nous aurions aimé créé une feature renseignant le numéro de cluster pour chaque document et une autre feature renseignant si oui ou non le commentaire appartient au même cluster que son parent. 

Par manque de temps, nous n'avons pu tester d'autres modèles (NMF, Kmeans, PLSA, etc).

Voici pour conclure les métriques permettant de mesure la qualité d'un modèle LDA. Tout seule elles n'ont pas vraiment d'intéret. Elles sont utiles lorsqu'on réalise plusieurs modèles avec des valeurs de k différentes. Effectivement, elles permettent de choisir la valeur de k optimum. 

In [ ]:
# Afficher le "Perplexity score"
# Temps d'exécution : 28 min 
print('\nPerplexity: ', lda_model.log_perplexity(corpus)) # a measure of how good the model is. lower the better.

# Afficher le "Coherence Score"
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)